In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity
import tkinter as tk


In [2]:
data = pd.read_csv('C:\\Users\\mithu\\Desktop\\VIT Projects\\food recomendation\\food-dataset (2).csv')

In [3]:
numerical_cols = ['sweet level', 'spice level', 'bitter level', 'sour level', 'salt level']
data[numerical_cols] = data[numerical_cols].fillna(data[numerical_cols].mean())
categorical_cols = ['temperature','taste']

In [4]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)])


In [5]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('kmeans', KMeans(n_clusters=5, random_state=42))])
pipeline.fit(data)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['sweet level', 'spice level',
                                                   'bitter level', 'sour level',
                                                   'salt level']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder())]),
                                                  ['temperature', 'taste'])])),
                ('kmeans', KMeans(n_clusters=5, random_state=42))])

In [6]:
cluster_labels = pipeline.named_steps['kmeans'].labels_

imputed_data = data.copy()
cluster_centroids = pipeline.named_steps['kmeans'].cluster_centers_

for i, cluster_label in enumerate(cluster_labels):
    if pd.isnull(imputed_data.iloc[i]['taste']):  
        imputed_data.at[i, 'taste'] = cluster_centroids[cluster_label, 0]  

In [7]:
print(imputed_data)
imputed_data.to_csv('updated_dataset.csv', index=False)

      Cuisine           Dish name                Dish   taste temperature  \
0     Chinese  panneer manchurian  Panneer Manchurian   Spicy         Hot   
1     Italian             lasagna             Lasagna   Spicy         Hot   
2      Indian         masala dosa         Masala Dosa   Spicy         Hot   
3    American               pizza               pizza   Salty         Hot   
4    American          cheesecake          Cheesecake   Sweet        Cold   
..        ...                 ...                 ...     ...         ...   
128    Indian       bharwa karela       Bharwa Karela  Bitter         Hot   
129   Indian        Methichi Usal       Methichi Usal  Bitter         Hot   
130    Indian          dried fish          dried fish   Salty         Hot   
131    Indian               papad               papad   Salty         Hot   
132  American             popcorn             popcorn   Salty         Hot   

     spice level  sweet level  salt level  sour level  bitter level  
0    

In [8]:
def recommend_dishes(user_input, metric='cosine', threshold=0.5):
    input_df = pd.DataFrame([user_input])
    
    processed_input = pipeline.named_steps['preprocessor'].transform(input_df)
    
    similarities = cosine_similarity(processed_input, pipeline.named_steps['preprocessor'].transform(data))
    distances = 1 - similarities
    
    closest_indices = np.argsort(distances)[0]
    
    filtered_indices = closest_indices[distances[0, closest_indices] < threshold]
    recommended_dishes = data.iloc[filtered_indices]
    
    recommended_dishes = recommended_dishes[(recommended_dishes['taste'] == user_input['taste']) & 
                                           (recommended_dishes['temperature'] == user_input['temperature']) &
                                           (recommended_dishes['Cuisine'] == user_input['cuisine'])]
    
    output_window = tk.Tk()
    output_window.title("Recommended Dishes")
    

    for index, row in recommended_dishes.iterrows():
        label = tk.Label(output_window, text=f"{row['Cuisine']}: {row['Dish name']} (Taste: {row['taste']}, Temperature: {row['temperature']})")
        label.pack()
    output_window.mainloop()
def get_user_input():
    root = tk.Tk()
    root.title("Dish Recommendation Input")

    tk.Label(root, text="Cuisine preference (e.g., Indian, Chinese, American):").grid(row=0, column=0, padx=10, pady=5)
    cuisine_entry = tk.Entry(root)
    cuisine_entry.grid(row=0, column=1, padx=10, pady=5)

    tk.Label(root, text="Taste preference (e.g., Sweet, Sour, Bitter):").grid(row=1, column=0, padx=10, pady=5)
    taste_entry = tk.Entry(root)
    taste_entry.grid(row=1, column=1, padx=10, pady=5)

    tk.Label(root, text="Temperature (e.g., Hot, Cold):").grid(row=2, column=0, padx=10, pady=5)
    temperature_entry = tk.Entry(root)
    temperature_entry.grid(row=2, column=1, padx=10, pady=5)

    tk.Label(root, text="Spice level (1-10):").grid(row=3, column=0, padx=10, pady=5)
    spice_entry = tk.Entry(root)
    spice_entry.grid(row=3, column=1, padx=10, pady=5)

    tk.Label(root, text="Sweet level (1-10):").grid(row=4, column=0, padx=10, pady=5)
    sweet_entry = tk.Entry(root)
    sweet_entry.grid(row=4, column=1, padx=10, pady=5)

    tk.Label(root, text="Salt level (1-10):").grid(row=5, column=0, padx=10, pady=5)
    salt_entry = tk.Entry(root)
    salt_entry.grid(row=5, column=1, padx=10, pady=5)

    tk.Label(root, text="Sour level (1-10):").grid(row=6, column=0, padx=10, pady=5)
    sour_entry = tk.Entry(root)
    sour_entry.grid(row=6, column=1, padx=10, pady=5)

    tk.Label(root, text="Bitter level (1-10):").grid(row=7, column=0, padx=10, pady=5)
    bitter_entry = tk.Entry(root)
    bitter_entry.grid(row=7, column=1, padx=10, pady=5)

    submit_button = tk.Button(root, text="Submit", command=lambda: process_user_input(root, cuisine_entry, taste_entry, temperature_entry, spice_entry, sweet_entry, salt_entry, sour_entry, bitter_entry))
    submit_button.grid(row=8, column=0, columnspan=2, pady=10)

    root.mainloop()
def process_user_input(root, cuisine_entry, taste_entry, temperature_entry, spice_entry, sweet_entry, salt_entry, sour_entry, bitter_entry):
    user_input = {
        'cuisine': cuisine_entry.get(),
        'taste': taste_entry.get(),
        'temperature': temperature_entry.get(),
        'spice level': float(spice_entry.get()),
        'sweet level': float(sweet_entry.get()),
        'salt level': float(salt_entry.get()),
        'sour level': float(sour_entry.get()),
        'bitter level': float(bitter_entry.get())
    }
    print("\nYou entered:")
    print(user_input)

    recommended_dishes_cosine = recommend_dishes(user_input, metric='cosine', threshold=0.5)
    print("\nRecommended Dishes (Cosine Similarity):\n", recommended_dishes_cosine)

get_user_input()



You entered:
{'cuisine': 'Indian', 'taste': 'Sour', 'temperature': 'Hot', 'spice level': 4.0, 'sweet level': 3.0, 'salt level': 6.0, 'sour level': 5.0, 'bitter level': 2.0}

Recommended Dishes (Cosine Similarity):
 None
